<a href="https://colab.research.google.com/github/flakesynto/MSc-Project/blob/main/MRI_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the prerequisites

In [ ]:
!pip install ultralytics==8.1.24

Reading and displaying the MRI Images with their Annotation Using COCO format

In [ ]:

import json
import os
import random
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import matplotlib
import numpy as np
import skimage
import tifffile
import os
import shutil
import skimage.draw
global mask_np
global filename

# Load the JSON file
with open(r'/content/drive/MyDrive/COCO/result.json', 'r') as file:
    data = json.load(file)
image_dir = r"/content/drive/MyDrive/COCO/images"
# Print out the first few image filenames/paths
for img in data['images'][:10]:  # Adjust the number to print more or fewer paths
    print(img['file_name'])


def display_images_with_coco_annotations(image_paths, annotations, display_type='both', colors=None):
    fig, axs = plt.subplots(2, 5, figsize=(15, 10))

    for ax, img_path in zip(axs.ravel(), image_paths):
        # Load image using OpenCV and convert it from BGR to RGB color space
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ax.imshow(image)
        ax.axis('on')  # Turn off the axes

        # Define a default color map if none is provided
        if colors is None:
            colors = matplotlib.colormaps['tab10']

        # Get image filename to match with annotations
        img_filename = os.path.basename(img_path)
        img_id = next(item['id'] for item in data['images'] if (item['file_name'].split('\\')[-1]) == img_filename)


        # Filter annotations for the current image
        img_annotations = [ann for ann in data['annotations'] if ann['image_id'] == img_id]


        for ann in img_annotations:
            category_id = ann['category_id']
            color = colors(category_id % 10)

            # Display bounding box
            if display_type in ['bbox', 'both']:
                bbox = ann['bbox']
                rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor=color, facecolor='none')
                ax.add_patch(rect)

            # Display segmentation polygon
            if display_type in ['seg', 'both']:
                for seg in ann['segmentation']:
                    poly = [(seg[i], seg[i+1]) for i in range(0, len(seg), 2)]
                    polygon = patches.Polygon(poly, closed=True, edgecolor=color, fill=True)
                    ax.add_patch(polygon)

    plt.tight_layout()
    plt.show()


# Get all image files
all_image_files = [os.path.join(image_dir, img['file_name'].split('\\')[-1]) for img in data['images']]
random_image_files = random.sample(all_image_files, 10)

# Choose between 'bbox', 'seg', or 'both'
display_type = 'seg'
display_images_with_coco_annotations(random_image_files, data, display_type)


def display_images_with_coco_annotations(image_paths, annotations, display_type='both', colors=None):
    fig, axs = plt.subplots(2, 5, figsize=(15, 10))

    for ax, img_path in zip(axs.ravel(), image_paths):
        # Load image using OpenCV and convert it from BGR to RGB color space
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ax.imshow(image)
        ax.axis('on')  # Turn off the axes

        # Define a default color map if none is provided
        if colors is None:
            colors = matplotlib.colormaps['tab10']

        # Get image filename to match with annotations
        img_filename = os.path.basename(img_path)
        img_id = next(item['id'] for item in data['images'] if (item['file_name'].split('\\')[-1]) == img_filename)


        # Filter annotations for the current image
        img_annotations = [ann for ann in data['annotations'] if ann['image_id'] == img_id]


        for ann in img_annotations:
            category_id = ann['category_id']
            color = colors(category_id % 10)

            # Display bounding box
            if display_type in ['bbox', 'both']:
                bbox = ann['bbox']
                rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor=color, facecolor='none')
                ax.add_patch(rect)

            # Display segmentation polygon
            if display_type in ['seg', 'both']:
                for seg in ann['segmentation']:
                    poly = [(seg[i], seg[i+1]) for i in range(0, len(seg), 2)]
                    polygon = patches.Polygon(poly, closed=True, edgecolor=color, fill=True)
                    ax.add_patch(polygon)

    plt.tight_layout()
    plt.show()


# Get all image files
all_image_files = [os.path.join(image_dir, img['file_name'].split('\\')[-1]) for img in data['images']]
random_image_files = random.sample(all_image_files, 10)

# Choose between 'bbox', 'seg', or 'both'
display_type = 'seg'
display_images_with_coco_annotations(random_image_files, data, display_type)

Displaying the MR Image Masks Using COCO format

In [ ]:
def create_mask(image_info, annotations, output_folder):
    global mask_np
    global filename
    # Create an empty mask as a numpy array
    mask_np = np.zeros((image_info['height'], image_info['width']), dtype=np.uint16)

    # Counter for the object number
    object_number = 1

    for ann in annotations:
        if ann['image_id'] == image_info['id']:
            # Extract segmentation polygon
            for seg in ann['segmentation']:
                # Convert polygons to a binary mask and add it to the main mask
                rr, cc = skimage.draw.polygon(seg[1::2], seg[0::2], mask_np.shape)
                mask_np[rr, cc] = object_number

                object_number += 1 #We are assigning each object a unique integer value (labeled mask)


    mask_np =  mask_np
    # Save the numpy array as a TIFF using tifffile library
    filename = image_info['file_name'].split('images\\')[-1]
    filename = filename.replace('.jpg','')
    filename = filename
    image_id = image_info['id']

    from skimage import measure

    # Visualize the mask
    plt.imshow(mask_np, cmap='nipy_spectral')
    plt.title(f"{filename}_mask_vis.png")

    # Turn off axis
    plt.axis('off')

    # Save visualized mask
    vis_file = os.path.join(output_folder, f"{filename}_mask_vis.png")
    plt.savefig(vis_file)
    # Show the plot
    plt.show()

    # Close plot
    plt.close()

    print(f"Saved visualized mask to {vis_file}")


def main(json_file, mask_output_folder, image_output_folder, original_image_dir):

    images = data['images']
    annotations = data['annotations']

    # Ensure the output directories exist
    if not os.path.exists(mask_output_folder):
        os.makedirs(mask_output_folder)
    if not os.path.exists(image_output_folder):
        os.makedirs(image_output_folder)

    for img in images:
        # Create the masks
        create_mask(img, annotations, mask_output_folder)

        # Copy original images to the specified folder
        original_image_path = os.path.join(original_image_dir, img['file_name'].split('\\')[-1])

        new_image_path = os.path.join(image_output_folder, os.path.basename(original_image_path))
        shutil.copy2(original_image_path, new_image_path)
        print(f"Copied original image to {new_image_path}")


if __name__ == '__main__':
    original_image_dir = image_dir  # Where your original images are stored
    json_file = data
    mask_output_folder = '/content/drive/MyDrive/COCO/masks'
    image_output_folder = '/content/drive/MyDrive/COCO/images_copied'
    main(json_file, mask_output_folder, image_output_folder, original_image_dir)

Reading and displaying the MR Images with their Annotation Using YOLO format

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as pltPolygon


# Function to read YOLO format annotations
def read_yolo_annotations(annotation_file):
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
        annotations = [line.strip().split() for line in lines]
    return annotations
# Function to create mask from polygons with different colors for each label
def create_mask(image_shape, annotations):
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    label_colors = {}  # Dictionary to store unique colors for each label

    for annotation in annotations:
        class_id = int(annotation[0])
        if class_id not in label_colors:
            label_colors[class_id] = tuple(np.random.randint(0, 256, size=(3,)))  # Generate random color for new label
        color = tuple(map(int, label_colors[class_id]))  # Convert color to tuple of integers

        points = np.array([float(val) for val in annotation[1:]], dtype=np.float32).reshape(-1, 2)
        points[:, 0] *= image_shape[1]  # Scaling x-coordinates
        points[:, 1] *= image_shape[0]  # Scaling y-coordinates
        points = points.astype(np.int32)
        cv2.fillPoly(mask, [points], color=color)  # Fill polygon with fully opaque white color

    return mask


# Set the paths to the data directories
data_dir = '/content/drive/MyDrive/YOLO_TEST'
images_dir = os.path.join(data_dir, 'images')
annotations_dir = os.path.join(data_dir, 'labels')

image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')]
annotation_files = [os.path.join(annotation_dir, os.path.splitext(os.path.basename(f))[0] + '.txt') for f in image_files]

# Load images
data_images = [cv2.imread(image_file) for image_file in image_files]
data_images = [cv2.cvtColor(data_image, cv2.COLOR_BGR2RGB) for data_image in data_images]

# Normalize pixel values to [0, 1]
normalized_images = [data_image / 255.0 for data_image in data_images]

# Display images with annotations, masked image, and mask side by side
for i, (image, annotation_file) in enumerate(zip(normalized_images, annotation_files)):
    annotations = read_yolo_annotations(annotation_file)
    mask = create_mask(image.shape, annotations)
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    # Apply the mask to the entire image

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot masked image
    axes[0].imshow(masked_image)
    axes[0].set_title('Masked Image')
    axes[0].axis('off')

    # Plot mask
    axes[1].imshow(mask, cmap='nipy_spectral')
    axes[1].set_title('Mask')
    axes[1].axis('off')

    plt.suptitle(f'Image {i+1}', fontsize=16)
    plt.show()

Spilt the YOLO format data into Train and Test dataset

In [ ]:
import os
import random
import cv2

def create_train_test_dirs(data_dir):
    """
    Create train and test directories for images and annotations.

    Args:
        data_dir (str): Path to the data directory.

    Returns:
        tuple: Paths to train and test directories for images and annotations.
    """
    train_dir = os.path.join(data_dir, 'train')
    test_dir = os.path.join(data_dir, 'test')

    train_images_dir = os.path.join(train_dir, 'images')
    train_annotations_dir = os.path.join(train_dir, 'labels')
    test_images_dir = os.path.join(test_dir, 'images')
    test_annotations_dir = os.path.join(test_dir, 'labels')

    for directory in [train_images_dir, train_annotations_dir, test_images_dir, test_annotations_dir]:
        os.makedirs(directory, exist_ok=True)

    return train_images_dir, train_annotations_dir, test_images_dir, test_annotations_dir

def split_and_save_data(images_dir, annotations_dir, train_images_dir, train_annotations_dir, test_images_dir, test_annotations_dir, split_ratio=0.8):
    """
    Split the data into train and test sets and save them in respective directories.

    Args:
        images_dir (str): Path to the directory containing images.
        annotations_dir (str): Path to the directory containing annotations.
        train_images_dir (str): Path to the directory to save train images.
        train_annotations_dir (str): Path to the directory to save train annotations.
        test_images_dir (str): Path to the directory to save test images.
        test_annotations_dir (str): Path to the directory to save test annotations.
        split_ratio (float): Ratio to split the data into train and test sets (default is 0.8).
    """
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]
    random.shuffle(image_files)
    split_index = int(split_ratio * len(image_files))

    for i, image_file in enumerate(image_files):
        image_path = os.path.join(images_dir, image_file)
        annotation_file = os.path.splitext(image_file)[0] + '.txt'
        annotation_path = os.path.join(annotations_dir, annotation_file)

        image = cv2.imread(image_path)

        with open(annotation_path, 'r') as f:
            annotations = f.readlines()

        if i < split_index:
            save_image_dir, save_annotation_dir = train_images_dir, train_annotations_dir
        else:
            save_image_dir, save_annotation_dir = test_images_dir, test_annotations_dir

        save_image_path = os.path.join(save_image_dir, image_file)
        save_annotation_path = os.path.join(save_annotation_dir, annotation_file)

        cv2.imwrite(save_image_path, image)
        with open(save_annotation_path, 'w') as f:
            f.writelines(annotations)

    print(f'Train set: {len(os.listdir(train_images_dir))} images, {len(os.listdir(train_annotations_dir))} annotations')
    print(f'Test set: {len(os.listdir(test_images_dir))} images, {len(os.listdir(test_annotations_dir))} annotations')



# Create train and test directories
train_images_dir, train_annotations_dir, test_images_dir, test_annotations_dir = create_train_test_dirs(data_dir)

# Split and save train and test data
split_and_save_data(images_dir, annotations_dir, train_images_dir, train_annotations_dir, test_images_dir, test_annotations_dir)

Training a pretrained YOLO Model for segmentation

***DO NOT RUN THIS CODE***

In [ ]:
from ultralytics import YOLO

def train_and_evaluate_YOLO(model_path, data_config_path, imgsz=512, batch=8, epochs=100):
    # Load the pre-trained model
    model = YOLO(model_path)

    # Train the model
    model.train(data=data_config_path, imgsz=imgsz, batch=batch, epochs=epochs, show_boxes=False)

    # Evaluate the model
    model.val(data=data_config_path)

# Paths and parameters
model_path = "/content/drive/MyDrive/YOLO/yolov8s-seg.pt"
data_config_path = "/content/drive/MyDrive/YOLO/data.yml"

# Train and evaluate YOLO model
train_and_evaluate_YOLO(model_path, data_config_path)

Copy the output of the model

In [ ]:
import os
import shutil

def copy_directory(source_dir, destination_dir):
    """
    Copy a single directory from the source to the destination.

    Args:
        source_dir (str): Path to the source directory.
        destination_dir (str): Path to the destination directory.
    """
    # Create destination directory if it does not exist
    os.makedirs(destination_dir, exist_ok=True)

    # Get the directory name from the source path
    source_dir_name = os.path.basename(source_dir)
    destination_path = os.path.join(destination_dir, source_dir_name)

    try:
        # Copy the entire directory
        shutil.copytree(source_dir, destination_path)
        print(f"Copied directory '{source_dir}' to '{destination_path}'")
    except shutil.Error as e:
        print(f"Error occurred while copying directory: {e}")
    except OSError as e:
        print(f"Error occurred while copying directory: {e}")

# Example usage
output_dir = '/content/drive/MyDrive/YOLO'
source_dir = '/content/runs'
copy_directory(source_dir, output_dir)

Predict the mask of a new MR Images using the Best Model

In [ ]:

import os
from google.colab.patches import cv2_imshow
import cv2

def predict_and_save_masks(model_path, source_dir, save=False):
    # Load the model
    model = YOLO(model_path)

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get a list of image files in the source directory
    image_files = [f for f in os.listdir(source_dir) if f.endswith('.jpg') or f.endswith('.png')]

    # Iterate over each image and predict the mask
    for image_file in image_files:
        image_path = os.path.join(source_dir, image_file)

        # Make prediction and save the output mask if specified
        if save:
            results = model.predict(source=image_path, save=True, show_boxes=False)  # Set project parameter to output_dir

        # Display the predicted image
        predicted_image_path = os.path.join('/content/runs/segment/predict', image_file)
        os.path.exists(predicted_image_path)
        predicted_image = cv2.imread(predicted_image_path)
        cv2_imshow(predicted_image)

# Set the paths
model_path = '/content/drive/MyDrive/YOLO_TEST/runs/segment/train3/weights/best.pt'
source_dir = '/content/drive/MyDrive/YOLO/test_predict'
output_dir = '/content/drive/MyDrive/YOLO/predict_output'

# Predict and display masks
predict_and_save_masks(model_path, source_dir, save=True)


def copy_files(source_dir, destination_dir):
    """
    Copy files from the source directory to the destination directory.

    Args:
        source_dir (str): Path to the source directory.
        destination_dir (str): Path to the destination directory.
    """
    # Create destination directory if it does not exist
    os.makedirs(destination_dir, exist_ok=True)

    # List all files in the source directory
    files = os.listdir(source_dir)

    # Copy each file from source to destination
    for file in files:
        source_file = os.path.join(source_dir, file)
        destination_file = os.path.join(destination_dir, file)
        shutil.copy(source_file, destination_file)
        print(f"Copied '{source_file}' to '{destination_file}'")

source_dir = '/content/runs/segment/predict'
copy_files(source_dir, output_dir)

Displaying the images with their predicted annotations

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
def display_masks(masks_dir):
    """
    Display the mask images without the underlying images.

    Args:
        masks_dir (str): Path to the directory containing mask images.
    """
    mask_files = [f for f in os.listdir(masks_dir) if f.endswith('.jpg') or f.endswith('.png')]

    for mask_file in mask_files:
        mask_path = os.path.join(masks_dir, mask_file)
        mask = cv2.imread(mask_path)
        cv2_imshow(mask)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Set the path to the directory containing mask images
masks_dir = '/content/drive/MyDrive/YOLO_TEST/predict_output/output'

# Display the mask images
display_masks(masks_dir)